In [1]:
import numpy as np
from ompl import base as ob
from ompl import geometric as og
from pydrake.planning import RobotDiagramBuilder
from pydrake.math import RigidTransform
from airo_drake import finish_build
import airo_models
from pydrake.visualization import AddFrameTriadIllustration
from pydrake.geometry import Meshcat, MeshcatVisualizer, MeshcatVisualizerParams, Role, GeometryFrame
from pydrake.math import RollPitchYaw, RotationMatrix
from pydrake.multibody.math import SpatialVelocity
from pydrake.common.eigen_geometry import Quaternion
import toppra as ta
import toppra.algorithm as algo
import toppra.constraint as constraint
from scipy.spatial.transform import Slerp
from scipy.spatial.transform import Rotation as R
from pydrake.all import Simulator


In [ ]:
# Create the robot diagram builder
robot_diagram_builder = RobotDiagramBuilder()

# Add meshcat visualizer
scene_graph = robot_diagram_builder.scene_graph()
builder = robot_diagram_builder.builder()

# Adding Meshcat must be done before finalizing
meshcat = Meshcat()
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Add visualizer for proximity/collision geometry
collision_params = MeshcatVisualizerParams(role=Role.kProximity, prefix="collision", visible_by_default=False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph.get_query_output_port(), meshcat, collision_params)

# Get plant and parser from the builder
plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()
parser.SetAutoRenaming(True)

# Create two cubes using airo_models
cube1_size = (0.2, 0.2, 0.2)
cube2_size = (0.2, 0.2, 0.2)
cube1_urdf_path = airo_models.box_urdf_path(cube1_size, "cube1")
cube2_urdf_path = airo_models.box_urdf_path(cube2_size, "cube2")

# Add cubes to the scene
cube1_index = parser.AddModels(cube1_urdf_path)[0]
cube2_index = parser.AddModels(cube2_urdf_path)[0]

cube1_body = plant.get_body(plant.GetBodyIndices(cube1_index)[0])
cube2_body = plant.get_body(plant.GetBodyIndices(cube2_index)[0])
AddFrameTriadIllustration(scene_graph=scene_graph, body=cube1_body)
AddFrameTriadIllustration(scene_graph=scene_graph, body=cube2_body)

# Position the cubes in space
world_frame = plant.world_frame()
cube1_frame = plant.GetFrameByName("base_link", cube1_index)
cube2_frame = plant.GetFrameByName("base_link", cube2_index)

# Set transforms for the cubes
cube1_transform = RigidTransform(p=[0.5, 0.0, 0.3])  # Position cube1 at (0.5, 0, 0.3)
cube2_transform = RigidTransform(p=[-0.5, 0.0, 0.3])  # Position cube2 at (-0.5, 0, 0.6)
robot_diagram, context = finish_build(robot_diagram_builder, meshcat)

# Place the cubes in the scene
plant_context = plant.GetMyContextFromRoot(context)
plant.SetFreeBodyPose(plant_context, cube1_body, cube1_transform)
plant.SetFreeBodyPose(plant_context, cube2_body, cube2_transform)
robot_diagram.ForcedPublish(context)

In [3]:
from pydrake.visualization import AddFrameTriadIllustration
from pydrake.math import RigidTransform, RollPitchYaw

# Create transforms for where you want the frames
goal_transform1 = RigidTransform(p=[0.3, 0.4, 0.5])
goal_transform2 = RigidTransform(p=[-0.3, 0.4, 0.5], rpy=RollPitchYaw(0, 0, np.pi/2))

# Add the triad visualizations using the world frame
world_frame = plant.world_frame()
AddFrameTriadIllustration(
    scene_graph=scene_graph,
    frame=world_frame,
    X_FT=goal_transform1,
    name="goal1"
)
AddFrameTriadIllustration(
    scene_graph=scene_graph,
    frame=world_frame,
    X_FT=goal_transform2,
    name="goal2"
)

# Make sure to publish to see the updates
robot_diagram.ForcedPublish(context)

In [4]:
def drake_transform_to_ompl_state(drake_transform: RigidTransform, ompl_state: ob.State):
    ompl_state.setX(drake_transform.translation()[0])
    ompl_state.setY(drake_transform.translation()[1])
    ompl_state.setZ(drake_transform.translation()[2])
    q = Quaternion(drake_transform.rotation().matrix())
    ompl_state.rotation().x = q.x()
    ompl_state.rotation().y = q.y()
    ompl_state.rotation().z = q.z()
    ompl_state.rotation().w = q.w()

In [ ]:
space = ob.CompoundStateSpace()
space.addSubspace(ob.SE3StateSpace(), 1.0)  # Left hand
space.addSubspace(ob.SE3StateSpace(), 1.0)  # Right hand

bounds = ob.RealVectorBounds(3)
robot_bounds = [-1.5, 1.5]
bounds.setLow(robot_bounds[0])
bounds.setHigh(robot_bounds[1])
space.getSubspace(0).setBounds(bounds)
space.getSubspace(1).setBounds(bounds)

problem = og.SimpleSetup(space)

# TODO: Implement state validity checker maybe basic collision avoidance
# problem.setStateValidityChecker(ob.StateValidityChecker(space))

start_state = ob.State(space)
drake_transform_to_ompl_state(plant.GetFreeBodyPose(plant.GetMyContextFromRoot(context), cube1_body), start_state()[0])
drake_transform_to_ompl_state(plant.GetFreeBodyPose(plant.GetMyContextFromRoot(context), cube2_body), start_state()[1])
problem.setStartState(start_state)

goal_state = ob.State(space)
drake_transform_to_ompl_state(goal_transform1, goal_state()[0])
drake_transform_to_ompl_state(goal_transform2, goal_state()[1])
problem.setGoalState(goal_state)
problem.setPlanner(og.RRTConnect(problem.getSpaceInformation()))
solved = problem.solve(5)
if solved:
    problem.simplifySolution()
    path_simplifier = og.PathSimplifier(problem.getSpaceInformation())
    path = problem.getSolutionPath()
    path_simplifier.smoothBSpline(path)

In [6]:
def plan_to_toppra(plan, vlims, alims):
    """Plan a path for the robot using TOPP-RA."""
    traj_s = np.linspace(0, 1, plan.shape[0])
    # Define the geometric path and two constraints.
    path = ta.SplineInterpolator(traj_s, plan)
    pc_vel = constraint.JointVelocityConstraint(vlims)
    pc_acc = constraint.JointAccelerationConstraint(alims)

    # We solve the parametrization problem
    instance = algo.TOPPRA(
        [pc_vel, pc_acc], path, parametrizer="ParametrizeConstAccel"
    )
    cart_traj = instance.compute_trajectory()

    # The output trajectory is an instance of
    # :class:`toppra.interpolator.AbstractGeometricPath`.
    ts_sample = np.linspace(
        0, cart_traj.duration, int(cart_traj.duration * 60)
    )
    qs_sample = cart_traj(ts_sample)
    qds_sample = cart_traj(ts_sample, 1)
    qdds_sample = cart_traj(ts_sample, 2)
    return ts_sample, qs_sample, qds_sample, qdds_sample

In [7]:
def dual_plan_to_numpy(plan):
    """Convert a dual arm plan to a numpy array."""
    left_traj = np.array(
        [
            np.array(
                [
                    plan.getState(i)[0].getX(),
                    plan.getState(i)[0].getY(),
                    plan.getState(i)[0].getZ(),
                    plan.getState(i)[0].rotation().x,
                    plan.getState(i)[0].rotation().y,
                    plan.getState(i)[0].rotation().z,
                    plan.getState(i)[0].rotation().w,
                ]
            )
            for i in range(plan.getStateCount())
        ]
    )
    right_traj = np.array(
        [
            np.array(
                [
                    plan.getState(i)[1].getX(),
                    plan.getState(i)[1].getY(),
                    plan.getState(i)[1].getZ(),
                    plan.getState(i)[1].rotation().x,
                    plan.getState(i)[1].rotation().y,
                    plan.getState(i)[1].rotation().z,
                    plan.getState(i)[1].rotation().w,
                ]
            )
            for i in range(plan.getStateCount())
        ]
    )
    return left_traj, right_traj

In [8]:
plan_left, plan_right = dual_plan_to_numpy(path)

dual_pos_plan = np.concatenate([plan_left[:, :3], plan_right[:, :3]], axis=1)

vlims = 0.2 * np.ones(dual_pos_plan.shape[1])
alims = 0.5 * np.ones(dual_pos_plan.shape[1])

ts_sample, pos, vel, acc = plan_to_toppra(dual_pos_plan, vlims, alims)
left_pos = pos[:, :3]
right_pos = pos[:, 3:]
left_vel = vel[:, :3]
right_vel = vel[:, 3:]
left_acc = acc[:, :3]
right_acc = acc[:, 3:]

slerp = Slerp(
    np.linspace(0, ts_sample[-1], plan_left.shape[0]),
    R.from_quat(plan_left[:, 3:7], scalar_first=False),
)
left_ori = slerp(ts_sample).as_quat()

slerp = Slerp(
    np.linspace(0, ts_sample[-1], plan_right.shape[0]),
    R.from_quat(plan_right[:, 3:7], scalar_first=False),
)
right_ori = slerp(ts_sample).as_quat()

In [9]:
meshcat.StartRecording(set_visualizations_while_recording=False, frames_per_second=60)

for i, t in enumerate(ts_sample):
    context.SetTime(t)
    plant_context = plant.GetMyContextFromRoot(context)
    q_left = Quaternion(left_ori[i])
    q_right = Quaternion(right_ori[i])
    plant.SetFreeBodyPose(plant_context, cube1_body, RigidTransform(p=left_pos[i], quaternion=q_left))
    plant.SetFreeBodyPose(plant_context, cube2_body, RigidTransform(p=right_pos[i], quaternion=q_right))
    robot_diagram.ForcedPublish(context)

meshcat.StopRecording()

In [10]:
meshcat.PublishRecording()

In [11]:
# Define grasp frame as middle point between the two cubes
cube1_pose = plant.GetFreeBodyPose(plant_context, cube1_body)
cube2_pose = plant.GetFreeBodyPose(plant_context, cube2_body)

grasp_point = cube1_pose.translation() + cube2_pose.translation()
grasp_point /= 2

In [23]:
# Move the grasp frame according to a specified velocity profile
velocity_time = np.linspace(0, 5, 100)
target_linear_velocity = np.array([0.1, 0, 0])
target_angular_velocity = np.array([0, 0, 0.1])


In [13]:
from scipy.linalg import expm
def skew(v):
    """Returns the skew-symmetric matrix of a 3-vector."""
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

In [24]:
meshcat.StartRecording(set_visualizations_while_recording=False, frames_per_second=60)
delta_t = velocity_time[1] - velocity_time[0]
for t in velocity_time:
    context.SetTime(t)
    plant_context = plant.GetMyContextFromRoot(context)
    cube1_pose = plant.GetFreeBodyPose(plant_context, cube1_body)
    cube2_pose = plant.GetFreeBodyPose(plant_context, cube2_body)
    cube1_velocity = target_linear_velocity + np.cross(target_angular_velocity, cube1_pose.translation() - grasp_point)
    cube2_velocity = target_linear_velocity + np.cross(target_angular_velocity, cube2_pose.translation() - grasp_point)
    cube1_pos = cube1_pose.translation() + cube1_velocity * delta_t
    cube2_pos = cube2_pose.translation() + cube2_velocity * delta_t
    grasp_point = grasp_point + target_linear_velocity * delta_t
    cube1_avel = cube1_pose.rotation().matrix() @ target_angular_velocity
    cube2_avel = cube2_pose.rotation().matrix() @ target_angular_velocity
    cube1_rot = cube1_pose.rotation() @ expm(skew(cube1_avel) * delta_t)
    cube2_rot = cube2_pose.rotation() @ expm(skew(cube2_avel) * delta_t)
    cube1_rot = RotationMatrix(cube1_rot)
    cube2_rot = RotationMatrix(cube2_rot)
    plant.SetFreeBodyPose(plant_context, cube1_body, RigidTransform(p=cube1_pos, R=cube1_rot))
    plant.SetFreeBodyPose(plant_context, cube2_body, RigidTransform(p=cube2_pos, R=cube2_rot))
    robot_diagram.ForcedPublish(context)

meshcat.StopRecording()

In [25]:
meshcat.PublishRecording()